# **INTRODUCTION**


*Définition du problème*

Dans le Défi de classification des questions de Quora Insincere, on a des questions et on nous demande de classer les questions comme sincères (0) ou non sincères (1). Le manque de sincérité dans ce cas comprend également des commentaires toxiques ou trompeurs, comme on peut le voir dans les échantillons ci-dessous. 
Dans les données de formation, on a l’identificateur de question (qid), la question (question_text) et la catégorie (target). Dans l’ensemble de tests, on a seulement l’identificateur et la question, et on nous demande de les classer dans les deux catégories.

# *Importation des données*

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import re
import matplotlib.pyplot as plt
import seaborn as sns

import math
import os
import time

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Flatten,Embedding,Activation, Dropout , Input
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D , LSTM, Bidirectional

from keras.models import Model
from keras import initializers, regularizers, constraints, layers

from tensorflow.keras.optimizers import Adam
from keras import optimizers, callbacks 
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
from keras.layers import Flatten
import sklearn.metrics as metrics

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [ ]:
# Lecture Data frame
train_df = pd.read_csv('../input/quora-insincere-questions-classification/train.csv')
test_df  = pd.read_csv('../input/quora-insincere-questions-classification/test.csv')

train_df.head()

In [ ]:
print(train_df.columns)

In [ ]:
train_df['question_text'].str.len().hist(color='y')

In [ ]:
f, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 4))

sns.countplot(train_df.target, palette=['blue', 'salmon'], ax=ax)

ax.set_title("Distribution des categories", fontsize=16)
ax.set_ylabel(ylabel='Count', fontsize=14)
ax.set_xticklabels(labels=['Sincere', 'Insincere'], fontsize=14)
ax.set_xlabel(xlabel='Category', fontsize=14)

plt.show()

In [ ]:
train_df["quest_len"] = train_df["question_text"].apply(lambda x: len(x.split()))

sincere = train_df[train_df["target"] == 0]
insincere = train_df[train_df["target"] == 1]

plt.figure(figsize = (15, 8))
sns.distplot(sincere["quest_len"], hist = True, label = "sincere")
sns.distplot(insincere["quest_len"], hist = True, label = "insincere")
plt.legend(fontsize = 10)
plt.title("Longueur des questions par Classe", fontsize = 12)
plt.show()

train_df["target"].value_counts()

In [ ]:
print("Train shape : ",train_df.shape)
print("Test shape : ",test_df.shape)

# Pré-traitement... une étape vers la vraie analyse

Nous sommes tous bien conscients que, avant que nous puissions utiliser des algorithmes de machine learning (ML) nous avons besoin de rendre notre dataset faisable pour l’analyse que on veut développer. Cette phase de pré-traitement des données est particulièrement importante lorsque on travaille avec des données en format texte. En effet, la plupart des mots qui constituent une phrase ne sont pas utiles au groupe de travail. L’objectif, à ce stade, est de rendre la vie de notre classeur aussi simple que possible afin d’en maximiser les performances. Voici quelques fonctions simples pour le nettoyage, non exhaustif, du texte :

In [ ]:
# remove space
spaces = ['\u200b', '\u200e', '\u202a', '\u202c', '\ufeff', '\uf0d8', '\u2061', '\x10', '\x7f', '\x9d', '\xad', '\xa0']
def remove_space(text):
    """
    remove extra spaces and ending space if any
    """
    for space in spaces:
        text = text.replace(space, ' ')
    text = text.strip()
    text = re.sub('\s+', ' ', text)
    return text


In [ ]:
# replace strange punctuations and raplace diacritics
from unicodedata import category, name, normalize

def remove_diacritics(s):
    return ''.join(c for c in normalize('NFKD', s.replace('ø', 'o').replace('Ø', 'O').replace('⁻', '-').replace('₋', '-'))
                  if category(c) != 'Mn')

special_punc_mappings = {"—": "-", "–": "-", "_": "-", '”': '"', "″": '"', '“': '"', '•': '.', '−': '-',
                         "’": "'", "‘": "'", "´": "'", "`": "'", '\u200b': ' ', '\xa0': ' ','،':'','„':'',
                         '…': ' ... ', '\ufeff': ''}
def clean_special_punctuations(text):
    for punc in special_punc_mappings:
        if punc in text:
            text = text.replace(punc, special_punc_mappings[punc])
    
    text = remove_diacritics(text)
    return text

**Expression régulière**, ou regex est extrêmement puissant dans la recherche et la manipulation de chaînes de texte, en particulier dans le traitement des fichiers texte. Une ligne de regex peut facilement remplacer plusieurs dizaines de lignes de codes de programmation.

In [ ]:
def clean_number(text):
    
    text = re.sub(r'(\d+)([a-zA-Z])', '\g<1> \g<2>', text)
    text = re.sub(r'(\d+) (th|st|nd|rd) ', '\g<1>\g<2> ', text)
    text = re.sub(r'(\d+),(\d+)', '\g<1>\g<2>', text)
    
    return text

In [ ]:
def decontracted(text):
    # specific
    text = re.sub(r"(W|w)on(\'|\’)t ", "will not ", text)
    text = re.sub(r"(C|c)an(\'|\’)t ", "can not ", text)
    text = re.sub(r"(Y|y)(\'|\’)all ", "you all ", text)
    text = re.sub(r"(Y|y)a(\'|\’)ll ", "you all ", text)

    # general
    text = re.sub(r"(I|i)(\'|\’)m ", "i am ", text)
    text = re.sub(r"(A|a)in(\'|\’)t ", "is not ", text)
    text = re.sub(r"n(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)re ", " are ", text)
    text = re.sub(r"(\'|\’)s ", " is ", text)
    text = re.sub(r"(\'|\’)d ", " would ", text)
    text = re.sub(r"(\'|\’)ll ", " will ", text)
    text = re.sub(r"(\'|\’)t ", " not ", text)
    text = re.sub(r"(\'|\’)ve ", " have ", text)
    return text

In [ ]:
import string
regular_punct = list(string.punctuation)
extra_punct = [
    ',', '.', '"', ':', ')', '(', '!', '?', '|', ';', "'", '$', '&','/', '[', ']', '>',
    '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£','·', '_', '{', '}', '©', '^', '`',
    '<', '→', '°', '€', '™', '›','♥', '←', '×', '§', '″', '′', 'Â', '½', 'à', '…', '“', '★',
    '”','–', '●', 'â', '►', '−', '¢', '²', '¶', '↑', '±', '¿', '▾','—', '‹', '─', '：', '¼', 
    '▼', '■', '’', '▀', '¨', '♫', '☆','é', '¯', '♦', '¤', '▲','è', '¸', '¾', 'Ã', '⋅', '‘', 
    '∙', '）','↓', '、', '│', '（', '»','，', '♪', '³', '・', '❤', 'ï', 'Ø', '≤', '√', '«', '»',
    '´', 'º', '¾', '¡', '§', '£', '₤']
all_punct = list(set(regular_punct + extra_punct))
# do not spacing - and .
all_punct.remove('-')
all_punct.remove('.')

def spacing_punctuation(text):
    """
    add space before and after punctuation and symbols
    """
    for punc in all_punct:
        if punc in text:
            text = text.replace(punc, f' {punc} ')
    return text

In [ ]:
train_df.question_text = train_df.question_text.apply(remove_space)
train_df.question_text = train_df.question_text.apply(clean_special_punctuations)
train_df.question_text = train_df.question_text.apply(clean_number)
train_df.question_text = train_df.question_text.apply(decontracted)
train_df.question_text = train_df.question_text.apply(spacing_punctuation)

train_df.head(5)

In [ ]:
test_df.question_text = test_df.question_text.apply(remove_space)
test_df.question_text = test_df.question_text.apply(clean_special_punctuations)
test_df.question_text = test_df.question_text.apply(clean_number)
test_df.question_text = test_df.question_text.apply(decontracted)
test_df.question_text = test_df.question_text.apply(spacing_punctuation)

test_df.head(5)

In [ ]:
## split to train and validation
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=1000)

In [ ]:
# Filling missing values in the text columns if any
train_X = train_df['question_text'].fillna("_na_").values
val_X   = val_df['question_text'].fillna("_na_").values
test_X  = test_df['question_text'].fillna("_na_").values

Maintenant, nous allons tokenize nos phrases et créer le vocabulaire. On peut définir un nombre max pour le nombre de mots dans le vocabulaire. 

In [ ]:
max_features = 50000 # how many unique words to use
maxlen       = 50  # max number of words in a question to use
embed_size   = 300 # how big is each word vector

# Tokenizing words in our sentences using keras tokenizer

tokenizer = Tokenizer(num_words = max_features)
tokenizer.fit_on_texts(list(train_X))

Pour prédire le prochain caractère, nous devons fournir au RNN une **séquence des caractères** précédents. Afin d’obtenir de nombreux exemples d’apprentissages, nous découpons des séquences dans les discours, en coulissant d’un certain pas entre chaque tranche.

In [ ]:
# converting each text in the dataset to a sequence of integers

train_X = tokenizer.texts_to_sequences(train_X)
val_X   = tokenizer.texts_to_sequences(val_X)
test_X  = tokenizer.texts_to_sequences(test_X)

In [ ]:
# Padding sequences 

train_X = pad_sequences(train_X, maxlen = maxlen)
val_X   = pad_sequences(val_X, maxlen = maxlen)
test_X  = pad_sequences(test_X, maxlen = maxlen)

In [ ]:
#Target values

train_y = train_df['target'].values
val_y = val_df['target'].values

In [ ]:
print(train_X[100])

# Chargement de l'Embeddings 

GloVe (« Global Vectors for Word Representation ») comme son nom l’indique est meilleur pour préserver les contextes globaux car il crée une matrice globale de co-occurrence en estimant la probabilité qu’un mot donné se produise avec d’autres mots.

D’abord il faut lire dans le fichier d'embedding dans un dictionnaire - chaque entrée est un mot, suivi du vecteur de nombres pour représenter ses valeurs.

In [ ]:
import zipfile

In [ ]:
with zipfile.ZipFile("../input/quora-insincere-questions-classification/embeddings.zip","r") as z:
    z.extractall(".")

In [ ]:
embeddings = '../input/glove840b300dtxt/glove.840B.300d.txt'
 
def get_coefs(word,*arr):
    return word, np.asarray(arr, dtype = 'float32')


In [ ]:
embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(embeddings))

all_embs = np.stack(embeddings_index.values())
emb_mean,emb_std = all_embs.mean(), all_embs.std()
embed_size = all_embs.shape[1]

word_index = tokenizer.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.random.normal(emb_mean, emb_std, (nb_words, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector
        

In [ ]:
embedding_matrix[3]

Le model BLSTM 

Il est possible d'utiliser un réseau appelé BLSTM, qui consiste à dédoubler une couche LSTM, l'une étant apprise pour parcourir le signal de gauche à droite, et l'autre de droite à gauche :
Les deux couches sont combinées afin de prendre les meilleures décisions locales en ayant "vu"... l'intégralité du signal !

ADAM est utile pour de l'optimisation stochastique. Elle utilise les EWA - Exponentially Weigthed Averages pour faire une estimation lissée des gradients à l'aide de moments

Le dropout est une technique qui est destinée à empêcher le sur-ajustement sur les données de training en abandonnant des unités dans un réseau de neurones. En pratique, les neurones sont soit abandonnés avec une probabilité pp ou gardés avec une probabilité 1-p1−p.

Un tel modèle s’implémente en quelques lignes avec Keras. On définit un modèle Sequential, sur lequel on ajoute une successions de couches qui correspondent directement aux étapes du schéma précédent. Les couches intermédiaires de dropout sont une technique de régularisation, pour éviter au modèle de trop coller aux données vues en entraînement et améliorer sa généralisation. Dans ce cas avec une probabilité d'etre abbandonné = 0.1

In [ ]:
inp = Input(shape=(maxlen,))
x = layers.Embedding(max_features, embed_size, weights=[embedding_matrix])(inp)
x = layers.Bidirectional(layers.LSTM(64, return_sequences=True))(x)
x = layers.Bidirectional(layers.LSTM(64))(x)
x = Dense(16, activation="relu")(x)
x = Dropout(0.1)(x)
x = Dense(1, activation="sigmoid")(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

# Entraîner un RNN à prédire
Notre objectif est de prédire, à partir d’une séquence de 50 caractères consécutifs, le caractère suivant. Il s’agit d’un problème d’apprentissage supervisé : à chaque itération, on fournit au modèle une séquence d’entrée encodée ainsi que le caractère encodé attendu en sortie. Le modèle effectue une prédiction, la compare à la cible attendue, et ajuste ses paramètres (aussi appelés poids du réseau) en cas d’erreur. 
Nous avons préparé nos données et créé notre modèle, il ne reste  maintenant plus qu’à lancer l’apprentissage. Une ligne suffit pour démarrer cette étape.

**ModelCheckpoint** 
A chaque époque, le pointeur de contrôle voit si les paramètres du modèle se sont améliorés - s’ils l’ont fait, il les enregistre dans un fichier appelé weights.hdf5.

In [ ]:
from keras.callbacks import ModelCheckpoint

In [ ]:
checkpointer = ModelCheckpoint(filepath="weights.hdf5", monitor='loss', save_best_only=True, mode='min')
history =model.fit(train_X, train_y, batch_size=512, epochs=5, callbacks=[checkpointer], validation_data=(val_X, val_y))

#history = model.fit(train_X, train_y, batch_size=512, epochs=5, validation_data=(val_X, val_y))

In [ ]:
print(history.history['loss'])
print(history.history['accuracy'])
print(history.history['val_loss'])
print(history.history['val_accuracy'])

**Epochs est le nombre maximum d’itérations ; 
batch_size correspond aux nombre d’observations que l’on fait passer avant de remettre à jour les poids synaptiques.
L’évolution de l’apprentissage est affichée dans la console IPython. En ce qui me concerne, voici
les valeurs finales de loss = 0.051 et accuracy = 0.955**

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model train vs validation loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper right')
plt.show()

Performance du modèle : fonction de perte liée à la phase d’entraînement et de test pour chaque epoch.

Sur le graphique, vous pouvez voir comment le modèle atteint un bon niveau de performance pendant la première Epoch.

thresh:	Il s’agit d’un entier qui spécifie le moins de valeurs non manquantes qui empêchent les lignes ou les colonnes de tomber.

La valeur de thresh est 2, ce qui signifie que nous évitons toute chute, au moins 2 valeurs non vides sont requises.

In [ ]:
import sklearn.metrics as accuracy_score 

In [ ]:
pred_glove_val_y = model.predict([val_X], batch_size=1024, verbose=1)
for thresh in np.arange(0.1, 0.501, 0.01):
    thresh = np.round(thresh, 2)
    print("F1 score at threshold {0} is {1}".format(thresh, metrics.f1_score(val_y, (pred_glove_val_y>thresh).astype(int))))

pred_glove_test_y = model.predict([test_X], batch_size=1024, verbose=1)

del word_index, embeddings_index, all_embs, embedding_matrix, model, inp, x
import gc; gc.collect()
time.sleep(10)

Score de soumission = 0,626 pas si mauvais !

In [ ]:
pred_test_y = pred_glove_test_y 
pred_test_y = (pred_test_y>0.35).astype(int) 
out_df = pd.DataFrame({"qid":test_df["qid"].values}) 
out_df['prediction'] = pred_test_y 
out_df.to_csv("submission.csv", index=False)

# Conclusions

Dans cette étude, une brève introduction a été offerte concernant l’utilisation de l’apprentissage profond inhérents à l’analyse du langage naturel. Tout d’abord, nous avons montré comment mettre en œuvre l’étape fondamentale du préprocessage des données. Dans cette section ont été discutées les principales techniques et méthodologies les plus communément utilisées, parmi lesquelles on cite le processus d'"analyse" afin de supprimer des parties de texte, de mots et de ponctuation qui ne sont pas utiles à l’accomplissement de la tâche de classement.
Ils suivent la procédure de "tokenizzazione" et de "padding" pour l’obtention d’un ensemble approprié de données utilisables par l’algorithme ML. Ensuite, la procédure d'"embedding" qui représente un pas fondamental et non négligeable pour l’amélioration et la représentation de l’ensemble de données dans un espace n-dimensionnel. À ce stade, le "pourquoi" de cette procédure est nécessaire et comment elle peut effectivement être mise en œuvre.
Les connaissances fournies par Glove sont alors transmises à notre modèle de réseau neural qui, par l’intermédiaire de la couche d’embedding, associe à chaque mot présent dans le dictionnaire un vecteur de taille @n'. Pour ce faire, cependant, il est nécessaire d’imposer la condition de non-remorquage de cette couche (trainable=False).
Une fois le modèle entraîné, on a procédé à l’évaluation du modèle et on a fourni le graphique relatif à la performance pendant la phase d’entraînement et le tableau relatif au rapport classification qui nous a permis d’évaluer la qualité de la méthodologie proposée.